In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# 1. Загрузка данных
# Пример данных: запросы и соответствующие ответы с метками релевантности
data = [
    {
        "query": "Добрый день! Подскажите пожалуйста, мне нужно создать заявку в директуме о переводе корпоративного номера тел на физ лицо в связи с увольнением,, я обновила директум, но не могу там найти как сделать эту заявку! Помогите пожалуйста оперативно. 12 декабря последний рабочий день мой. Спасибо",
        "answers": [
            {
                "text": "Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке:",
                "label": 1,
            },
            {"text": "Оформите обращение в команду поддержки Directum:", "label": 0},
            {
                "text": "Для передачи оборудования оформите заявку по ссылке:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Коллеги, добрый день! Клиент как обычно скинул заказы в PDF, но они почему то теперь не читаются: Обработать заказы нужно срочно, поэтому заявка через Элму не актуальна. Цена вопроса около 20 млн. Помогите, пжл, разобраться, что случилось. Проблема не только на моей стороне, так же читаются файлы и у моей коллеги. С ее слов, на прошлой неделе у нее была такая же проблема, но она успела обработать заказы. Позже она зашла, чтоб проверить и файлы стали нечитабельны.",
        "answers": [
            {
                "text": "Для получения поддержки по программному обеспечению заполните заявку по ссылке:",
                "label": 1,
            },
            {
                "text": "Для обслуживания принтера заполните заявку по ссылке:",
                "label": 0,
            },
            {
                "text": "Оформите обращение в команду поддержки Directum:",
                "label": 0,
            },
            {
                "text": "Поднимите данный запрос, если необходимо разработать новый ЗНО, доработать уже существующее решение на платформе ELMA365",
                "label": 0,
            },
        ],
    },
    {
        "query": "Новая учетная запись nestlesoft Здравствуйте. Меняла пароль самостоятельно до 19 декабря, не могу зайти ни по одному паролю. Сделала запрос на сброс пароля.\r\n\r\n \r\n\r\nBest regards, \r\n\r\nLiubov Talalaeva\r\n\r\nTrainee\r\n\r\nNestlé Rossiya LLC\r\n\r\nCell: +79184192478\r\n\r\nPlease note that starting from January 1, 2025, my corporate email address – Liubov.Talalaeva@nestle.ru <mailto:Liubov.Talalaeva@nestle.ru> \r\n\r\n \r\n\r\n\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Уточните номер заявки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": " ПАРОЛЬ ДЛЯ АККАУНТА С НОВЫМ ДОМЕНОМ Коллеги, добрый день!\r\n\r\nПожалуйста, помогите получить пароль для учетной записи с новым доменом. \r\nЯ поднимала тикет, но мне никакого смс с новым паролем не приходило.\r\n\r\n\r\nРуководитель в отпуске, поэтому новый запрос отправить и быстро одобрить не получится.\r\n\r\nХЕЛП\r\n\r\n \r\n\r\nС уважением,\r\n\r\n \r\n\r\nАНАСТАСИЯ ЧАРНУШИНА\r\n\r\nJunior Brand Manager PRO PLAN Dog\r\nNestle Purina PetСare Russia\r\n\r\n \r\n\r\nTel. / 8 915 577 29 28\r\n\r\n* / anastasiia.charnushina@ru.nestle.com <mailto:anastasiia.charnushina@ru.nestle.com>  \r\n\r\n\r\n\r\n \r\n\r\nOut of the office on:\r\n\r\n∙ Dec 23 – Dec 28\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Уточните номер заявки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Re: не удается поменять пароль Муса добрый день!\r\n\r\nПароль соответствует всем требованиям. \r\nЧерез CTRL+ ALT + DELETE я меняю пароль на текущем домене. Я задавал вопрос по замене пароля на новом домене, на который переезжаем с нового года. Согласно высланной инструкции.\r\n\r\n\r\nИли я что то делаю не так?\r\n\r\nПолучите Outlook для Android <https://eur02.safelinks.protection.outlook.com/?url=https%3A%2F%2Faka.ms%2FAAb9ysg&data=05%7C02%7CService.Desk%40ru.nestle.com%7Cc13cd877d5c14c37257a08dd241cd7d5%7C12a3af23a7694654847f958f3d479f4a%7C0%7C0%7C638706428484848763%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=0JxGC4ZCsB8Wy%2Fc1CTkkiRGlhGpkDukcE7EoIULfR3M%3D&reserved=0> \r\n________________________________\r\n\r\nFrom: Dzhamalutdinov,Musa,RU-Moscow <Musa.Dzhamalutdinov@ru.nestle.com>\r\nSent: Tuesday, December 24, 2024 3:52:21 PM\r\nTo: RU: Service Desk, RUSSIA <Service.Desk@ru.nestle.com>; Riabenkii,Sergei,RU-St-Petersburg <Sergei.Riabenkii@ru.nestle.com>\r\nSubject: RE: не удается поменять пароль \r\n \r\n\r\nСергей, добрый день. \r\n\r\n \r\n\r\nПодскажите, а пробовали менять пароль через CTRL+ ALT + DELETE? \r\n\r\nПароль содержит спец.символы, цифры и нижний и верхний регистр? \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\nBest Regards,\r\n\r\nMusa Dzhamalutdinov\r\n\r\nIT Specialist\r\n\r\nInformation Technology Bureau I Moscow \r\n\r\n \r\n\r\nOffice: +7 495 725 7222 <tel:+74957257222> /+7 495 568 0122 <tel:+74957257222>  \r\n\r\nEmail: Musa.Dzhamalutdinov@ru.nestle.com <mailto:Musa.Dzhamalutdinov@ru.nestle.com> \r\n\r\n \r\n\r\n \r\n\r\nFrom: RU: Service Desk, RUSSIA <Service.Desk@ru.nestle.com> \r\nSent: Tuesday, December 24, 2024 3:29 PM\r\nTo: Riabenkii,Sergei,RU-St-Petersburg <Sergei.Riabenkii@ru.nestle.com>; RU: Service Desk, RUSSIA <Service.Desk@ru.nestle.com>; Dzhamalutdinov,Musa,RU-Moscow <Musa.Dzhamalutdinov@ru.nestle.com>\r\nSubject: RE: не удается поменять пароль\r\n\r\n \r\n\r\nДобрый день,\r\n\r\n@Dzhamalutdinov,Musa,RU-Moscow <mailto:Musa.Dzhamalutdinov@ru.nestle.com> , помоги, пожалуйста.\r\n\r\n \r\n\r\n \r\n\r\n\r\n\r\nBest Regards,\r\n\r\nRoman \r\n\r\nUser support division manager\r\n\r\nInformation Technology Bureau I Moscow \r\n\r\n \r\n\r\nOffice: +7 495 725 7222 <tel:+74957257222> /+7 495 568 0122 <tel:+74957257222>  \r\n\r\nWeb: nestle.service-now.com <https://eur02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fnestle.service-now.com%2Fsp&data=05%7C02%7CService.Desk%40ru.nestle.com%7Cc13cd877d5c14c37257a08dd241cd7d5%7C12a3af23a7694654847f958f3d479f4a%7C0%7C0%7C638706428484872753%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=9l0uPjjZLevr4hLo54Mauq6K8NZVjuyz3PHP4lYveP8%3D&reserved=0>   \r\n\r\nEmail: service.desk@ru.nestle.com <mailto:service.desk@ru.nestle.com>  \r\n\r\nFAQ: Частые вопросы в Service Desk <https://eur02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fnestle.service-now.com%2Fsp%3Fid%3Dnestle_kb_article_view%26sysparm_article%3DKB0068377%26sys_kb_id%3Dcc48fe25db0d211052a729560596199a%26spa%3D1&data=05%7C02%7CService.Desk%40ru.nestle.com%7Cc13cd877d5c14c37257a08dd241cd7d5%7C12a3af23a7694654847f958f3d479f4a%7C0%7C0%7C638706428484890577%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=LPBBoKQEtY8berbHuLPR%2BklhF%2FzRciDvUgu10fZrguU%3D&reserved=0> \r\n\r\n \r\n\r\n \r\n\r\nFrom: Riabenkii,Sergei,RU-St-Petersburg <Sergei.Riabenkii@ru.nestle.com <mailto:Sergei.Riabenkii@ru.nestle.com> > \r\nSent: Tuesday, December 24, 2024 8:32 AM\r\nTo: RU: Service Desk, RUSSIA <Service.Desk@ru.nestle.com <mailto:Service.Desk@ru.nestle.com> >\r\nSubject: не удается поменять пароль\r\n\r\n \r\n\r\nКоллеги,\r\n\r\n \r\n\r\nне удается поменять пароль\r\n\r\n\r\n\r\nSergei Riabenki\r\n\r\nNational Sales Manager, Russia \r\n\r\n  \r\n\r\nSredniy prospect V.O., 36/40 litera A, Business Centre Ostrov, 7 floor\r\n\r\nSaint-Petersburg, Russia 199004 \r\n\r\nOffice +7 812 315 9585\r\n\r\nMobile +7 921 958 20 96\r\n\r\n\r\n\r\n \r\n\r\n \r\n",
        "answers": [
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "пароль от новой локлальной учетной записи Здравствуйте,\r\n\r\nЯ на новой локальной учётной записи заменяла пароль, теперь он не работает, как поступить?\r\n\r\n \r\n\r\nBest regards,\r\n\r\nOlga Ivanova.\r\n\r\nMarketing Manager Coffee\r\n\r\nCell: +79162932855\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Доступ в Директум This message is from an EXTERNAL SENDER. BE CAUTIOUS, particularly with links and attachments. \r\n________________________________\r\n\r\n\r\nКоллеги, добрый день.\r\n\r\n \r\n\r\nНе могу зайти в директум (использую логин и пароль как для винды), а мне он срочно нужен.\r\n\r\n \r\n\r\nПодскажите, пожалуйста, что можно сделать?\r\n\r\n \r\n\r\nСпасибо,\r\n\r\nКсения\r\n\r\n \r\n\r\nKseniia Moshonkina\r\n\r\nD2D Manager | Nestle Purina PetCare Russia\r\n\r\nMobile Phone: +7 916 036 01 51 | \r\n\r\nAddress: 2/1 Paveletskaya square, 115054 Moscow, Russia |\r\n\r\nwww.purina.ru <https://eur02.safelinks.protection.outlook.com/?url=http%3A%2F%2Fwww.purina.ru%2F&data=05%7C02%7CService.Desk%40ru.nestle.com%7C9763fcd26fb649ca313408dd24b09a12%7C12a3af23a7694654847f958f3d479f4a%7C0%7C0%7C638707063127384707%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=SKWk3U%2BnNeQMAEUSVv%2FgkHE%2FWY8iPsbgslAj2nNPBCk%3D&reserved=0> \r\n\r\n \r\n",
        "answers": [
            {
                "text": "Оформите обращение в команду поддержки Directum:",
                "label": 1,
            },
            {
                "text": "Для получения доступа к Консультант Плюс оформите заявку по ссылке:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Не работает Директум 5.6 Коллеги, здравствуйте!\r\n\r\n \r\n\r\nОчень нужна ваша консультация. Не могу войти в систему Directum 5.6, хотя ВПН подключён. \r\n\r\n\r\n\r\nС уважением,\r\n\r\nСветлана\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Оформите обращение в команду поддержки Directum:",
                "label": 1,
            },
            {
                "text": "Для получения доступа к Консультант Плюс оформите заявку по ссылке:",
                "label": 0,
            },
        ],
    },
    {
        "query": "НЕ работает принтер (РСО Иркутск)  \r\n\r\nДобрый день.\r\n\r\n \r\n\r\nВ Иркутском офисе принтер не выводит документы на печать.\r\n\r\n \r\n\r\nПробовали отправлять на печать с разных ноутбуков, перезагружали, бумага и тонер есть, к сети подключен.\r\n\r\n \r\n\r\nНикаких ошибок не выдает.\r\n\r\n \r\n\r\nПрошу проверить, в чем проблема.\r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\nEkaterina Matvienko\r\n\r\nRSO Assistant\r\n\r\nNestle Rossiya, LLC\r\n\r\n6 Lenina str, office 306\r\n\r\n664025 Irkutsk \r\n\r\n+79149265011\r\n\r\n \r\n\r\n \r\n",
        "answers": [
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для обслуживания принтера заполните заявку по ссылке:",
                "label": 0,
            },
        ],
    },
    {
        "query": "пароль от Nestlesoft Коллеги, добрый день,\r\n\r\n \r\n\r\nПосле получения нового пароля от Nestlesoft, я не могу зайти в локальную учетку. Новый пароль не работает.\r\n\r\nВчера я подняла заявку в Elma REQ0012296, но со мной никто не связался.\r\n\r\nПожалуйста, дайте корректный пароль от Nestlesoft, чтобы я могла продолжить работу. \r\n\r\n \r\n\r\nСпасибо.\r\n\r\n \r\n\r\nАлёна Зевина \r\n\r\nСтарший аналитик\r\n\r\nM: +7 916 462 42 69\r\n\r\nООО Нестле Россия\r\n\r\n115054, Россия, Москва, Павелецкая пл., д.2 стр. 1\r\n\r\n\r\n\r\nПожалуйста, примите во внимание, что с 01.01.2025 адрес моей корпоративной электронной почты – alena.zevina@nestle.ru <mailto:alena.zevina@nestle.ru> \r\n\r\n \r\n\r\n \r\n",
        "answers": [
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Не пускает в новую учетку  Коллеги, добрый день!\r\n\r\n \r\n\r\nМеня не пускает в office учетку. \r\n\r\nНи с тем паролем, который мне на телефон высылали, ни с тем, что было в письме от 19 числа.\r\n\r\n \r\n\r\nПодскажите, что делать?\r\n\r\n \r\n\r\nBest regards,\r\n\r\n\r\nAbasova Elena\r\nDemand & Supply Planning Manager | Nestlé Purina PetCare Russia\r\nMobile: +7 (965) 416-54-51\r\nE-mail: elena.abasova@ru.nestle.com <mailto:elena.abasova@ru.nestle.com> \r\nAddress: 2/1 Paveletskaya square, 115054 Moscow, Russia\r\n\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "не обновляются данные с сервера в файле на OneDrive Коллеги, добрый день\r\n\r\n \r\n\r\nПовторяется ошибка синхронизации файла\r\n\r\n \r\n\r\nТакая ошибка уже была ранее\r\n\r\nПришлось скачивать версию с сервера и заново переразмещать у себя на One Drive (так как файл хранится у меня)\r\n\r\nСегодня снова та же самая ошибка\r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\nЧто можно сделать?\r\n\r\n \r\n\r\nСпасибо,\r\n\r\nКатя \r\n\r\n \r\n\r\nPlease be noted that starting from 01/01/2025 my corporate email changes to ekaterina.potashnikova@nestle.ru <mailto:ekaterina.potashnikova@nestle.ru> \r\n\r\n \r\n\r\nEkaterina Potashnikova \r\n\r\nIMS Projects and Customer Service Export Manager\r\n\r\n \r\n\r\nNestle Rossiya LLC\r\n\r\n115054, 2/1 Paveletskaya sqr, Moscow, Russia \r\n\r\nM +7 916 814-3847\r\n\r\n  \r\n\r\n\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Для получения поддержки по программному обеспечению заполните заявку по ссылке:",
                "label": 1,
            },
            {
                "text": "Для решения проблемы с IT-оборудованием обратитесь по ссылке:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Не получается зайти в Microsoft c новой почты  \r\n\r\n \r\n\r\n \r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\n \r\n\r\n\r\n\r\n \r\n\r\n \r\n\r\nПожалуйста, примите во внимание, что с 01.01.2025 адрес моей корпоративной электронной почты: Mariia.ukolova@nestle.ru\r\n\r\n \r\n\r\nMaria Ukolova|Procurement Business Partner|Nestle Russia| mobile phone: +7 9057241694\r\n\r\n \r\n",
        "answers": [
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Использование некорпоративных мессенджеров запрещено.",
                "label": 0,
            },
            {
                "text": "При выявлении ошибок в работе робота (макроса) оформите обращение по ссылке:",
                "label": 0,
            },
        ],
    },
    {
        "query": "восстановление пароля удаленно Добрый день!\r\n\r\n \r\n\r\nПодскажите, что нужно сделать сотруднику на удаленке, у которого просрочился пароль и он не может войти в систему? \r\n\r\n \r\n\r\nBest regards,\r\n\r\nSvetlana Chiveleva |Raw Materials Sourcing Manager|Nestle Russia\r\n\r\n2/1 Paveletskaya Sq 115054 Moscow| www.nestle.com <http://www.nestle.com/> \r\n\r\n\r\n\r\nПожалуйста, примите во внимание, что с 01.01.2025 адрес моей корпоративной электронной почты – svetlana.chiveleva@nestle.ru <mailto:svetlana.chiveleva@nestle.ru>  \r\n\r\nPlease be informed that starting from 01.01.2025 my e-mail will be – svetlana.chiveleva@nestle.ru <mailto:svetlana.chiveleva@nestle.ru> \r\n\r\n \r\n\r\n \r\n",
        "answers": [
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "ЭП не подписывает в Гамильтон Добрый день, не могу подписать ЭП в гамильтон",
        "answers": [
            {
                "text": "Для получения поддержки по программному обеспечению заполните заявку по ссылке:",
                "label": 1,
            },
            {
                "text": "Поднимите данный запрос, если необходимо разработать новый ЗНО, доработать уже существующее решение на платформе ELMA365",
                "label": 0,
            },
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 0,
            },
        ],
    },
    {
        "query": "Директум Коллеги, добрый день,      Прошу разблокировать учетную запись.",
        "answers": [
            {
                "text": "Оформите обращение в команду поддержки Directum:",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 0,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Блокировка учетки Коллеги, Добрый день\r\n\r\nПодскажите как разблокировать учетку в SAP APO? Слишком много раз ввела неправильно пароль\r\n\r\nЗаранее Спасибо",
        "answers": [
            {
                "text": "Для сброса пароля в Local SAP Prod (R9*) заполните заявку ниже:",
                "label": 1,
            },
            {
                "text": "Для сброса пароля в Local SAP non-Prod (R5*, R8*) заполните заявку ниже:",
                "label": 1,
            },
            {
                "text": "Приложите скриншот ошибки, пожалуйста.",
                "label": 0,
            },
        ],
    },
    {
        "query": "Изменение лимита на диске N Вы не часто получаете электронную почту от anna.gavrilkina@nestle.ru. Узнайте, почему это важно\r\nThis message is from an EXTERNAL SENDER. BE CAUTIOUS, particularly with links and attachments.\r\n________________________________\r\nКоллеги, добрый день!\r\n\r\nПыт",
        "answers": [
            {
                "text": "Чтобы изменить тип лицензии на вашей учетной записи оформите заявку ниже:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Нет доступа к настольным приложениям Office 365",
        "answers": [
            {
                "text": "Чтобы изменить тип лицензии на вашей учетной записи оформите заявку ниже:",
                "label": 1,
            },
            {
                "text": "Оформите обращение в команду поддержки Directum:",
                "label": 0,
            },
        ],
    },
    {
        "query": "Добрый день! вторую неделю не могу зайти на страницу я и нестле Идентификатор взаимосвязи: 41cf7da1-0026-b000-a717-cae5800ec94a Дата и время: 02.02.2025 23:47:22 Пользователь: elizaveta.makeeva@nestle.ru Тип проблемы: У пользователя нет разрешений.",
        "answers": [
            {
                "text": "Для получения поддержки по программному обеспечению заполните заявку по ссылке:",
                "label": 1,
            },
            {
                "text": "Чтобы изменить тип лицензии на вашей учетной записи оформите заявку ниже:",
                "label": 0,
            },
            {
                "text": "Оформите обращение в команду поддержки Directum:",
                "label": 0,
            },
        ],
    },
    
]

# 2. Подготовка данных для обучения
train_examples = []
for item in data:
    query = item["query"]
    for answer in item["answers"]:
        train_examples.append(
            InputExample(texts=[query, answer["text"]], label=answer["label"])
        )

In [ ]:
# 3. Загрузка предобученной модели
model = SentenceTransformer("./FRIDA")

# 4. Настройка и дообучение модели
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=6)
train_loss = losses.OnlineContrastiveLoss(model)

epochs = 5
# Дообучение модели
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=epochs,
    warmup_steps = int(len(train_dataloader) * 0.1),
    show_progress_bar=True,
    optimizer_params={'lr': 1e-5},
    scheduler='warmupconstant',
    weight_decay=0.01,
    checkpoint_path='./checkpoints',
)

c:\Users\RUShulpiAn.NESTLESOFT\Desktop\ServiceBot\.conda\Lib\site-packages\datasets\table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
c:\Users\RUShulpiAn.NESTLESOFT\Desktop\ServiceBot\.conda\Lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Step,Training Loss


In [3]:
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer("./frida_model")
# 5. Оценка модели
# Пример оценки модели на новых данных
test_query = [
    "Коллеги, добрый день! Клиент как обычно скинул заказы в PDF, но они почему то теперь не читаются: Обработать заказы нужно срочно, поэтому заявка через Элму не актуальна. Цена вопроса около 20 млн. Помогите, пжл, разобраться, что случилось. Проблема не только на моей стороне, так же читаются файлы и у моей коллеги. С ее слов, на прошлой неделе у нее была такая же проблема, но она успела обработать заказы. Позже она зашла, чтоб проверить и файлы стали нечитабельны.",
    "Добрый день! Подскажите пожалуйста, мне нужно создать заявку в директуме о переводе корпоративного номера тел на физ лицо в связи с увольнением,, я обновила директум, но не могу там найти как сделать эту заявку! Помогите пожалуйста оперативно. 12 декабря последний рабочий день мой. Спасибо",
]
test_answers = [
    "Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке:",
    "Оформите обращение в команду поддержки Directum:",
    "Для передачи оборудования оформите заявку по ссылке:",
    "Для получения поддержки по программному обеспечению заполните заявку по ссылке:",
]

answer_embeddings = model.encode(test_answers)

for i in range(2):
    query_embedding = model.encode(test_query[i])
    # Вычисление косинусного сходства
    cosine_scores = cosine_similarity([query_embedding], answer_embeddings)[0]

    # Сортировка ответов по сходству
    sorted_answers = sorted(
        zip(test_answers, cosine_scores), key=lambda x: x[1], reverse=True
    )
    print(f"Question: {test_query[i]}")
    # Вывод отсортированных ответов
    for answer, score in sorted_answers:
        print(f"Answer: {answer}, score: {score}")
    print()

Question: Коллеги, добрый день! Клиент как обычно скинул заказы в PDF, но они почему то теперь не читаются: Обработать заказы нужно срочно, поэтому заявка через Элму не актуальна. Цена вопроса около 20 млн. Помогите, пжл, разобраться, что случилось. Проблема не только на моей стороне, так же читаются файлы и у моей коллеги. С ее слов, на прошлой неделе у нее была такая же проблема, но она успела обработать заказы. Позже она зашла, чтоб проверить и файлы стали нечитабельны.
Answer: Для передачи оборудования оформите заявку по ссылке:, score: 0.38137882947921753
Answer: Для получения поддержки по программному обеспечению заполните заявку по ссылке:, score: 0.3522518277168274
Answer: Оформите обращение в команду поддержки Directum:, score: 0.3483220040798187
Answer: Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке:, score: 0.22275573015213013

Question: Добрый день! Подскажите пожалуйста, мне нужно создать заявку в директуме о переводе к

In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./FRIDA")

df = pd.read_excel("TEMPLATES.xlsx")
new_answers = df['Текст'].tolist()
answer_embeddings = model.encode(new_answers)

total_score = 0
for test in data:
    query = test['query']
    answers = test['answers']

    query_embedding = model.encode(query)
    cosine_scores = cosine_similarity([query_embedding], answer_embeddings)[0]
    sorted_indices = np.argsort(cosine_scores)[::-1][:3]
    
    score = 0
    print(query.replace("\r\n", " "))
    for num, index in enumerate(sorted_indices):
        for i in answers:
            if i["text"] == new_answers[index]:
                if i["label"] == 1:
                    if num == 0:
                        score += 2
                    else:
                        score += 1
        print(new_answers[index], score)
        break

    total_score += score
    print(f'Результат: {score}\n')

print(f'Общая сумма результатов: {total_score}')

NameError: name 'pd' is not defined

In [ ]:
model.save("./shul_er_v6")

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]